<a href="https://colab.research.google.com/github/9-coding/Google_ML_Bootcamp_5th/blob/main/NL2SQL_gemma2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
!pip install --quiet datasets
!pip install --quiet huggingface_hub
!pip install --quiet peft
!pip install --quiet trl
!pip install --quiet bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGr

In [25]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load Dataset

In [11]:
dataset = load_dataset("Clinton/Text-to-sql-v1")
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text'],
        num_rows: 262208
    })
})

In [12]:
dataset['train'][0]

{'instruction': 'Name the home team for carlton away team',
 'input': 'CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n)',
 'response': 'SELECT home_team FROM table_name_77 WHERE away_team = "carlton"',
 'source': 'sql_create_context',
 'text': 'Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: Name the home team for carlton away team ### Input: CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n) ### Response: SELECT home_team FROM table_name_77 WHERE away_team = "carlton"'}

# QLoRA

In [13]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load Gemma 2

In [16]:
BASE_MODEL = "google/gemma-2-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Prompt Format
```
<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model
```

In [17]:
text = dataset['train']['text'][0]
input = dataset['train']['input'][0]
instruction = dataset['train']['instruction'][0]
stripped_text = text[:text.find("### Instruction:")].strip()
content = f"{stripped_text}\n{instruction}\n{input}"

print(content)

Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.
Name the home team for carlton away team
CREATE TABLE table_name_77 (
    home_team VARCHAR,
    away_team VARCHAR
)


In [18]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

messages = [
    {
        "role": "user",
        "content": content
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [19]:
prompt

'<bos><start_of_turn>user\nBelow are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.\nName the home team for carlton away team\nCREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n)<end_of_turn>\n<start_of_turn>model\n'

In [20]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

In [21]:
print(outputs[0]["generated_text"][len(prompt):])

```sql
CREATE TABLE table_name_77 (
    home_team VARCHAR,
    away_team VARCHAR
);
``` 

**Explanation:**

* **CREATE TABLE table_name_77:** This statement creates a new table named "table_name_77". 
* **home_team VARCHAR:** This defines a column named "home_team" that will store text values (strings) in the format of team names.
* **away_team VARCHAR:** This defines a column named "away_team" that will store text values (strings) in the format of team names.

**Important:** 

* You need to replace "table_name_77" with a meaningful name for your table. 
* This code creates a table with two columns, one for the home team and one for the away team. 


Let me know if you have any other SQL tasks you'd like to perform! 



In [22]:
def generate_prompt(example):
  prompt_list = []
  for i in range(len(example['text'])):
    text = dataset['train']['text'][i]
    input = dataset['train']['input'][i]
    instruction = dataset['train']['instruction'][i]
    stripped_text = text[:text.find("### Instruction:")].strip()
    content = f"{stripped_text}\n{instruction}\n{input}"

    prompt_list.append(r"""<bos><start_of_turn>user{}<end_of_turn>
      <start_of_turn>model
      {}<end_of_turn><eos>""".format(content, example['response'][i]))
    return prompt_list

In [23]:
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>userBelow are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.
Name the home team for carlton away team
CREATE TABLE table_name_77 (
    home_team VARCHAR,
    away_team VARCHAR
)<end_of_turn>
      <start_of_turn>model
      SELECT home_team FROM table_name_77 WHERE away_team = "carlton"<end_of_turn><eos>


# Trainer Settings

In [27]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_ratio=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/262208 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [28]:
trainer.train()

It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Step,Training Loss


KeyboardInterrupt: 